In [122]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
import utils

In [123]:
trafo_id = "T1330" #T1330 T0932
data, _ = utils.read_raw_network_data(trafo_id, depth=1)

In [124]:
processed_data = utils.preprocess(data)

ts_data = processed_data["measurements"]
nodes_data = processed_data["nodes_static_data"]
edges_data = processed_data["edges_static_data"]


In [125]:
#maybe add this to utils
nodes_data = pd.get_dummies(nodes_data, columns=["aclass_id"])

In [126]:
nodes_data.head()

,node_id,smm_list,prikljucna_moc_odjem,prikljucna_moc_odjem_aggr,prikljucna_moc_oddaja,prikljucna_moc_oddaja_aggr,x_y,lon_lat,aclass_id_PMO,aclass_id_RO,aclass_id_TR,aclass_id_junction
0,0,[nan],[nan],NaN,[nan],NaN,"(530670.5350000001, 153984.66459999979)","(21.489888056885366, 49.254464475525744)",0,0,1,0
1,1,[nan],[nan],NaN,[nan],NaN,"(530624.3164999997, 154054.6559999995)","(21.48928277111593, 49.25509150386318)",0,1,0,0
2,2,[nan],[nan],NaN,[nan],NaN,"(530614.5626999997, 153929.52500000037)","(21.48916671919818, 49.25396514965445)",0,1,0,0
3,3,[nan],[nan],NaN,[nan],NaN,"(530582.3731000004, 153957.00579999946)","(21.488746936005448, 49.2542105251013)",0,1,0,0
4,4,[nan],[nan],NaN,[nan],NaN,"(530534.8115999997, 153926.6462999992)","(21.48813262185447, 49.253934624161346)",0,1,0,0


In [127]:
edges_data.head()

,from_node_id,to_node_id,length_km,r_ohm_per_km,x_ohm_per_km,max_i_ka,type,d_mm2,coords,coords_transformed
0,94,110,0.055,0.533,0.079,0.176,cs,70.0,"[[530374.6612999998, 153999.8621999994], [5303...","[[21.486049387812564, 49.254584020619056], [21..."
1,100,110,0.037,0.533,0.079,0.176,cs,70.0,"[[530337.4517000001, 154015.33029999956], [530...","[[21.485565488389135, 49.2547210147785], [21.4..."
2,103,110,0.038,1.040,0.081,0.120,cs,35.0,"[[530337.4517000001, 154015.33029999956], [530...","[[21.485565488389135, 49.2547210147785], [21.4..."
3,103,104,0.035,0.627,0.081,0.159,cs,35.0,"[[530331.6523000002, 154036.7006000001], [5303...","[[21.48548847348933, 49.25491294229187], [21.4..."
4,6,7,0.092,0.533,0.079,0.176,cs,70.0,"[[530465.5811999999, 153943.5175999999], [5304...","[[21.487233318527778, 49.25408238978174], [21...."


In [128]:
ts_data.head()

,date_time,node_id,voltage,temperature_2m,snow_depth,cloud_cover,is_day,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,active_power,reactive_power
0,2021-06-01 00:00:00,0.0,234.276667,13.055012,0.0,20.1,0.0,0.0,0.0,0.0,0.0,168900.000,29000.0
1,2021-06-01 00:00:00,8.0,234.433333,13.037499,0.0,20.1,0.0,0.0,0.0,0.0,0.0,0.212,0.0
2,2021-06-01 00:00:00,9.0,232.700000,13.076500,0.0,20.1,0.0,0.0,0.0,0.0,0.0,1.304,0.0
3,2021-06-01 00:00:00,10.0,232.850000,13.076500,0.0,20.1,0.0,0.0,0.0,0.0,0.0,0.236,0.0
4,2021-06-01 00:00:00,11.0,232.833333,13.076500,0.0,20.1,0.0,0.0,0.0,0.0,0.0,0.132,0.0


### StaticGraphTemporalSignal

##### Parameters
* edge_index (Numpy array) – Index tensor of edges.

* edge_weight (Numpy array) – Edge weight tensor.

* features (Sequence of Numpy arrays) – Sequence of node feature tensors.

* targets (Sequence of Numpy arrays) – Sequence of node label (target) tensors.

* **kwargs (optional Sequence of Numpy arrays) – Sequence of additional attributes.

edge_index

In [129]:
edge_index = edges_data[["from_node_id", "to_node_id"]].values.T
edge_index.shape

(2, 114)

edge_weight

In [130]:
#maybe change this later in utils.py
# edges_data_dummies = pd.get_dummies(edges_data, columns=["type"])
# edges_data_dummies.head()
#edge type is only one? drop it
edges_data = edges_data.drop(columns=["type"])

In [131]:
edge_weight = edges_data.drop(["from_node_id", "to_node_id"], axis=1).values
edge_weight.shape

(114, 7)

features (node)

In [142]:
#create an empty array of shape (num_of_ts, node_features,num_nodes)
num_of_ts = len(ts_data[ts_data["node_id"] == 0])
num_nodes = len(nodes_data["node_id"].unique())
#create an empty array of length 
num_of_static_features = len(nodes_data.columns) - 1 #exclude node_id
num_of_dynamic_features = len(ts_data.columns) - 2 #exclude node_id and timestamp
node_features = np.empty((num_of_ts,num_nodes,num_of_static_features + num_of_dynamic_features))
node_features.shape

(70160, 113, 33)

In [146]:
merged_data = pd.merge(ts_data, nodes_data, on='node_id', how='left')
merged_data.head()

,date_time,node_id,voltage,temperature_2m,snow_depth,cloud_cover,is_day,shortwave_radiation,direct_radiation,diffuse_radiation,...,prikljucna_moc_odjem_y,prikljucna_moc_odjem_aggr_y,prikljucna_moc_oddaja_y,prikljucna_moc_oddaja_aggr_y,x_y_y,lon_lat_y,aclass_id_PMO_y,aclass_id_RO_y,aclass_id_TR_y,aclass_id_junction_y
0,2021-06-01 00:00:00,0.0,234.276667,13.055012,0.0,20.1,0.0,0.0,0.0,0.0,...,[nan],NaN,[nan],NaN,"(530670.5350000001, 153984.66459999979)","(21.489888056885366, 49.254464475525744)",0,0,1,0
1,2021-06-01 00:00:00,8.0,234.433333,13.037499,0.0,20.1,0.0,0.0,0.0,0.0,...,[35.0],35.0,[0.0],0.0,"(530715.1489000004, 154094.69480000064)","(21.490457517273, 49.25545698477883)",1,0,0,0
2,2021-06-01 00:00:00,9.0,232.700000,13.076500,0.0,20.1,0.0,0.0,0.0,0.0,...,[17.0],17.0,[14.0],14.0,"(530714.4113999996, 153845.96270000003)","(21.49046867801605, 49.25321912788421)",1,0,0,0
3,2021-06-01 00:00:00,10.0,232.850000,13.076500,0.0,20.1,0.0,0.0,0.0,0.0,...,[14.0],14.0,[0.0],0.0,"(530699.8213, 153848.67300000042)","(21.490279227137176, 49.253242668676066)",1,0,0,0
4,2021-06-01 00:00:00,11.0,232.833333,13.076500,0.0,20.1,0.0,0.0,0.0,0.0,...,[14.0],14.0,[0.0],0.0,"(530683.6710999999, 153849.89320000075)","(21.49006966676341, 49.25325271264873)",1,0,0,0


In [159]:
ro_junction_nodes = nodes_data[(nodes_data['aclass_id_RO'] == 1) | (nodes_data['aclass_id_junction'] == 1)]['node_id']
node_types = ["aclass_id_RO", "aclass_id_junction","aclass_id_PMO","aclass_id_TR"]

In [160]:
num_of_time_series = len(ts_data[ts_data["node_id"] == 0])
num_of_nodes = len(nodes_data)
num_of_static_features = len(nodes_data.columns) - 1 #exclude node_id
num_of_dynamic_features = len(ts_data.columns) - 2  # Exclude 'node_id' and 'timestamp'

result_array = np.zeros((num_of_time_series, num_of_nodes, num_of_static_features + num_of_dynamic_features))
result_array.shape

(70160, 113, 33)

In [172]:
# Populate the array with static and dynamic features
for i, node_id in enumerate(nodes_data['node_id']):
    # Check if the node is of type 'RO' or 'junction'
    if node_id in ro_junction_nodes.values:
        # For 'RO' and 'junction', fill dynamic features with zeros
        result_array[:, i, num_of_static_features:] = 0
    else:
        # For other nodes, fill with static and dynamic features
        node_data = merged_data[merged_data['node_id'] == node_id]
        result_array[:, i] = node_data.drop(['node_id', 'date_time'], axis=1).values


ValueError: setting an array element with a sequence.

targets (time series)

In [ ]:
target = []

### Create a signal

The signal is needed for the GNN

In [173]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

In [ ]:
dataset = StaticGraphTemporalSignal(
            edge_index, edge_weight, node_features, target
        )